# IEooc_Methods5_Exercise4a: Calculating income specific footprints for Germany


**Prerequisites**
- Basic knowledge of Input-Output Analysis (completed part Methodology 5: Input-output analysis of the IEooc)
- Basic knowledge on Python (see here for help https://simschul.github.io/python_basics/python_cheatsheet.html)
- Basic knowledge (and an installation) of the Pymrio python package (read https://pymrio.readthedocs.io/en/latest/intro.html#)
- EXIOBASE for the year 2013 in the product by product variant (pxp) (you can find it here: https://zenodo.org/records/5589597 , download the file “IOT_2013_pxp.zip” and save it to your computer)

## Import libraries

In [2]:
# import libraries
import pandas as pd
import numpy as np
import pymrio
import os
from plotnine import *
# TODO: add other libraries you need (e.g. matplotlib)


## Specify paths

It's good practice to specify all paths at the beginning of your script so that other people who want to run your script directly know which lines to adapt to run the script in their own computer. 

In [11]:
path2final_demand = #TODO: add path to the folder where you have stored the 'Final_demand_by_income_avg.xlsx' file
path2exiobase = #TODO: add path to folder where you have stored the exiobase .zip file from Zenodo 

In [ ]:
# Define the path to the Excel file containing final demand by income data
path2file = os.path.join(path2final_demand, 'Final_demand_by_income_avg.xlsx')

# Read the Excel file into a DataFrame, setting the first two columns as the index
Y_by_income = #TODO 

# Set the name of the columns to 'income_group'
#TODO

# Display the DataFrame
Y_by_income

## Step 4: Import Exiobase

In [13]:
# Parse the Exiobase3 data from the specified path using the pymrio library
exio = #TODO 

Now, let's calculate all missing parts from our MRIO system (L, S, ...):

In [ ]:
#TODO: calculate missing parts from MRIO system using pymrio functions

## Step 5: Calculate total footprints by income

$ Footprint = S * L * Y $

Check dimensions of the three matrices first: 

In [ ]:
print("S: ", exio.impacts.S.shape)
print("L: ", exio.L.shape)
print("Y: ", Y_by_income.shape)


Do the matrix multiplications: 

In [16]:
D_by_income = #TODO

In [ ]:
D_by_income

### a) Extract CO2, CH4 and N2O footprints in GHGeq

We can extract the specific rows from `D_by_income` using .loc and specifing the entire name of the impact. E.g. for CO2 this would be:

In [ ]:
D_by_income.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)']

If we don't want to always write the long names of the impacts we can also search for keywords using the following code:

In [ ]:
# Searching for all impacts that contain CO2EQ
D_by_income.loc[D_by_income.index.str.contains('CO2EQ')]

Or we search using regular expressions:

In [ ]:
# searching for all impacts that contain CH4 and CO2EQ
D_by_income.loc[D_by_income.index.str.contains(r'(?=.*CH4)(?=.*CO2EQ)')]

We extract the data we need to plot from `D_by_income` and store it as new variable `data2plot`. Then you need to prepare the data so that you can pass it to your favorite plot library (this step is very much dependend on which library you use for plotting). 

In [21]:
# Extract the CO2, CH4 and N2O footprints measured in CO2 equivalents
data2plot = #TODO

# Prepare the data for plotting
#TODO



### b) Visualize income specific footprints

Plot the income specific footprints. For inspiration of you can check out different plot types here: https://www.python-graph-gallery.com/

In [ ]:
#TODO: plot the data

## Step 7: Break down emissions by final demanded product

To break down emissions by final product we need to calculate $D = S*L*\hat{Y}$. However, in our case $Y$ is already a matrix, thus we cannot diagnoalise it as we did when $Y$ when was just a vector. 
One option we have is to calculate the broken down emissions **seperatedely** for each income group. Thus, we can extract the respective column from Y for each income group, convert it to a vector and then diagnolise it. This is best done within a for loop: 

(Note: These calculation might take some time. To see the progress you can print the income group for each iteration.)

In [ ]:
# Create empty list to store results

D_by_income_and_fd_list = #TODO: create an empty list

for income_group in Y_by_income.columns: 
    #TODO: print progress (as this takes a while to run): 
    
    # extract the respective column of final demand and convert it to a numpy array:
    y_vector =  #TODO
    
    # diagnoalise this vector and convert to pd DataFrame:
    Y_diag = #TODO
    
    #TODO: calculate footprints and append to result list
    

    

Now we can bind the list elements together to one large dataframe using `pd.concat`, again specifing the keys and axis (axis=1 means column-wise): 

In [ ]:
D_by_income_and_fd = #TODO
D_by_income_and_fd

`D_by_income_and_fd` has now three dimensional columns for the dimensions: income group, sector and country. The first column of this dataframe, for example, contains all emissions related to the final consumption of Austrian Paddy Rice by the average income group (Group00).  

To further analyse these results we convert this dataframe again from the wide to the long format. If you have never heard about these two data formats, have a look here: https://towardsdatascience.com/reshaping-a-pandas-dataframe-long-to-wide-and-vice-versa-517c7f0995ad 


In [ ]:
df = #TODO
df

We exclude all zero values to save some memory: 

In [31]:
df = #TODO

It's often more convenient to have the variables (e.g. income group, impacts type, sector, region) as **indices** instead of individual columns, whereas the values (column: `value`) are of course better placed as individual column. To put the variables as MultiIndex we use the `set_index` method: 

In [32]:
df.set_index(keys=['impact', "income_group", "region", "sector"], inplace=True)

Now, we select CO2 emissions using the `.loc` method. NOte: we can only use .loc because we set the variables as Index beforehand.  

In [33]:
df_co2 = df.loc['Carbon dioxide (CO2) CO2EQ IPCC categories 1 to 4 and 6 to 7 (excl land use, land use change and forestry)']

Next, we want to know for each income group for how much CO2 emissions each finally demanded product is responsible. Thus, we sum over income groups and sectors: 

In [ ]:
df_co2_by_fd = #TODO
df_co2_by_fd

Then, we can calcualte the shares of each sector of total emissions by income group (that's why we first group by income group): 

In [ ]:
df_co2_by_fd['share'] =  #TODO
df_co2_by_fd

Since we are interested in the 3 largest contributors by income group we rank the emissions (again first grouped by income group) using the `rank` method, and assigning the ranks as a column. 

Then we can filter the dataframe by ranks smaller than 4: 

In [ ]:
df_co2_by_fd['rank'] = #TODO
df_co2_by_fd[df_co2_by_fd['rank'] < 4]

#TODO: What are the top 3 products (summed over all regions) that contribute most  for each income group? 